# 📌 Estrutura do Notebook 02C-Ingestao-Bronze-EstatisticaJogadorPorPartida
📍 Objetivo: Ler os dados da camada Bronze, aplicar transformações e salvar na Silver.

In [0]:
%run "/Users/caio.santos.cavalheiro@gmail.com/00-Configuracao"

Leitura das credenciais de forma segura pelo Spark 🔐


Caminhos definidos:
📂 Bronze: s3a://mvp-brasileirao-2024/bronze/ | 📂 Silver: s3a://mvp-brasileirao-2024/silver/ | 📂 Gold: s3a://mvp-brasileirao-2024/gold/


## 🔶 Leitura dos dados da camada Bronze 

In [0]:
# Importar bibliotecas necessárias
from pyspark.sql import SparkSession

# Definir o caminho do arquivo CSV na Bronze
csv_file_path = f"s3a://{aws_bucket_name}/bronze/BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv"

# Ler o arquivo CSV da camada Bronze
df_bronze = spark.read.csv(csv_file_path, header=True, inferSchema=True, sep=";", encoding='utf-8')

print(f'✅ Leitura do Arquivo BrasilSerieA_2024_EstatisticaJogadorPorPartida na camada Bronze feita com sucesso ')

✅ Leitura do Arquivo BrasilSerieA_2024_EstatisticaJogadorPorPartida na camada Bronze feita com sucesso 



## 🔶 Aplicar transformações (limpeza e padronização) 

Limpezas que serão aplicadas no arquivo `BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv`

#### 🔹 Transforma a coluna 'Cmp%' STRING → DOUBLE

In [0]:
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import DoubleType

# Removendo possíveis caracteres extras (como % ou espaços) e convertendo para Double
df_bronze = df_bronze.withColumn("Cmp%", regexp_replace(col("Cmp%"), "[^0-9.]", "").cast(DoubleType()))

print(f'✅ Campo "Cmp%" atualizado para Double')

✅ Campo "Cmp%" atualizado para Double


#### 🔹 Remove Colunas desnecessárias

In [0]:
# Removendo colunas com PySpark
df_bronze = df_bronze.drop('npxG')
print(f'✅ Colunas removidas com sucesso!')

✅ Colunas removidas com sucesso!


#### 🔹 Renomeia as Colunas do Dataframe

In [0]:
df_bronze = df_bronze.withColumnRenamed("Time", "Clube") \
                     .withColumnRenamed("#", "Numero_Camisa") \
                     .withColumnRenamed("Na��o", "Nacionalidade") \
                     .withColumnRenamed("Pos.", "Posicao_em_Campo") \
                     .withColumnRenamed("Min.", "Minutos_em_Campo") \
                     .withColumnRenamed("Assis.", "Assitencias") \
                     .withColumnRenamed("PB", "Penaltis_Batidos") \
                     .withColumnRenamed("PT", "Penaltis_Tentados") \
                     .withColumnRenamed("TC", "Total_Chutes") \
                     .withColumnRenamed("CaG", "Chutes_a_Gol") \
                     .withColumnRenamed("CrtsA", "Cartao_Amarelo") \
                     .withColumnRenamed("CrtV", "Cartao_Vermelho") \
                     .withColumnRenamed("Contatos", "Toques_na_Bola") \
                     .withColumnRenamed("Div", "Desarmes") \
                     .withColumnRenamed("Crts", "Interceptacoes") \
                     .withColumnRenamed("xG", "Gols_Esperados") \
                     .withColumnRenamed("xAG", "Assistencias_Esperadas") \
                     .withColumnRenamed("SCA", "Acoes_de_Criacao") \
                     .withColumnRenamed("GCA", "Acoes_de_Gol") \
                     .withColumnRenamed("Cmp", "Passes_Completos") \
                     .withColumnRenamed("Att", "Passes_Tentados") \
                     .withColumnRenamed("Cmp%", "Prct_Passes_Certos") \
                     .withColumnRenamed("PrgP", "Passes_Progressivos") \
                     .withColumnRenamed("Condu��es", "Conducoes_de_Bola") \
                     .withColumnRenamed("PrgC", "Conducoes_Progressivas") \
                     .withColumnRenamed("Tent", "Dribles_Tentados") \
                     .withColumnRenamed("Suc", "Dribles_Completos")

print(f'✅ Colunas Renomeadas atendendo a Padronização!')

✅ Colunas Renomeadas atendendo a Padronização!


#### 🔹 Split na coluna 'Idade' para pegar os 2 primeiros caracteres

In [0]:
from pyspark.sql.functions import col, substring

# Mantendo apenas os dois primeiros caracteres da coluna "Idade"
df_bronze = df_bronze.withColumn("Idade", substring(col("Idade"), 1, 2))

print(f'✅ Split na coluna Idade para pegar os 2 primeiros caracteres!')

✅ Split na coluna Idade para pegar os 2 primeiros caracteres!


#### ⚠️ Ajuste no Jogador 'Felipe Vieira' (PARTICULARIDADE)
Idade faltante na base original. Confirmado em fonte externa oficial

In [0]:
from pyspark.sql.functions import when, col

# Lista com as datas específicas em que faltou o dado
datas_sem_idade = ['2024-04-13', '2024-04-28', '2024-05-04', '2024-05-13', '2024-07-07', '2024-07-28']

# Critério específico para preencher somente o jogador e datas corretas
df_bronze = df_bronze.withColumn(
    'idade',
    when(
        (col('Jogador') == 'Felipe Vieira') &
        (col('Data').isin(datas_sem_idade)) &
        (col('idade').isNull()),
        22
    ).otherwise(col('idade'))
)

print('⚠️ INFORMAÇÃO IMPORTANTE SOBRE TRATAMENTO DOS DADOS!!')
print('    Jogador: Felipe Vieira | Clube: Fluminense | Campo Preenchido: [Idade]')
print(f'    Data das Partidas: {datas_sem_idade}')
print('    Motivo: Idade faltante na base original. Confirmado em fonte externa oficial')
print('    Valor Preenchido = 22 anos')
print(f'    FONTE: https://www.ogol.com.br/jogador/felipe-andrade/620537?epoca_id=153')
print(f'✅ Feita o preenchimento da Informação!')

⚠️ INFORMAÇÃO IMPORTANTE SOBRE TRATAMENTO DOS DADOS!!
    Jogador: Felipe Vieira | Clube: Fluminense | Campo Preenchido: [Idade]
    Data das Partidas: ['2024-04-13', '2024-04-28', '2024-05-04', '2024-05-13', '2024-07-07', '2024-07-28']
    Motivo: Idade faltante na base original. Confirmado em fonte externa oficial
    Valor Preenchido = 22 anos
    FONTE: https://www.ogol.com.br/jogador/felipe-andrade/620537?epoca_id=153
✅ Feita o preenchimento da Informação!


#### 🔹 Preenche campos que estão NULLS com cada critério

In [0]:
# Preenche 'Nacionalidade' com 'N/D', já que esse campo possui 3 Caracteres, para mantermos o padrão
df_bronze = df_bronze.fillna({'Nacionalidade': 'N/D'})

# Preenche 'Minutos_em_Campo' com '0', já que o campo é INT
df_bronze = df_bronze.fillna({'Minutos_em_Campo': 0})

# Preencher automaticamente todos os campos numéricos vazios com zero
df_bronze = df_bronze.fillna(0)

print(f'✅ Preenche as colunas vazias de acordo com os devidos critérios!')

✅ Preenche as colunas vazias de acordo com os devidos critérios!


#### 🔹Dicionário de padronização dos nomes dos clubes

In [0]:
# Dicionário de padronização dos nomes dos clubes (em ordem alfabética)
mapeamento_clubes = {
    "Atlético Goianiense": "Atlético-GO",
    "Atlético Mineiro": "Atlético-MG",
    "Athletico Paranaense": "Athletico-PR",
    "Bahia": "Bahia",
    "Botafogo (RJ)": "Botafogo",
    "Red Bull Bragantino": "Bragantino",
    "Corinthians": "Corinthians",
    "Criciuma": "Criciúma",
    "Cruzeiro": "Cruzeiro",
    "Cuiaba": "Cuiabá",
    "Flamengo RJ": "Flamengo",
    "Fluminense": "Fluminense",
    "Fortaleza": "Fortaleza",
    "Gremio": "Grêmio",
    "Internacional": "Internacional",
    "Juventude": "Juventude",
    "Palmeiras": "Palmeiras",
    "Sao Paulo": "São Paulo",
    "Vasco": "Vasco da Gama",
    "Vitória": "EC Vitória"
}

df_bronze = df_bronze.replace(mapeamento_clubes, subset=["Clube"])

print(f'✅ Dicionário de padronização dos nomes dos clubes (em ordem alfabética)!')

✅ Dicionário de padronização dos nomes dos clubes (em ordem alfabética)!


#### 🔹Nova coluna para identificar a Origem dos dados

In [0]:
from pyspark.sql.functions import lit

# Criar a coluna 'Origem_Dado' no df_bronze (dados originais)
df_bronze = df_bronze.withColumn("Origem_Dado", lit("Original"))

print(f'✅ Criação de uma coluna para identificar Origem do Dado!')

✅ Criação de uma coluna para identificar Origem do Dado!


#### ⚠️ Ajuste manual das partidas que não foram computadas na base de dados

In [0]:
from pyspark.sql.functions import to_date, col

csv_input_manual = f"s3a://{aws_bucket_name}/bronze/BrasilSerieA_2024_EstatisticaJogador_InputManual.csv"

df_input_manual = spark.read.csv(csv_input_manual, header=True, inferSchema=True, sep=";", encoding='utf-8')

df_input_manual = df_input_manual.withColumn("Origem_Dado", lit("Input Manual"))

df_input_manual = df_input_manual.withColumn("Data", to_date(col("Data"), "dd/MM/yyyy"))

print('⚠️ INFORMAÇÃO IMPORTANTE SOBRE TRATAMENTO DOS DADOS!!')
print('    Clube: Fluminense   | Data da Partida: 01/11/2024')
print('    Clube: Grêmio       | Data da Partida: 01/11/2024')
print('    Clube: Atlético-MG  | Data da Partida: 07/12/2024')
print('    Clube: Athletico-PR | Data da Partida: 07/12/2024')
print('    Motivo: Partidas não computadas na base original. Confirmado em fonte externa oficial.')
print('    Os valores preenchidos estão na Coluna [Origem_Dado] com o valor: "Input Manual" ')
print(f'    FONTE: https://fbref.com/pt/partidas/bb1a4828/Fluminense-Gremio-2024Novembro1-Serie-A')
print(f'    FONTE: https://fbref.com/pt/partidas/fd4de3a4/Atletico-Mineiro-Athletico-Paranaense-2024Dezembro8-Serie-A')
print(f'✅ Feita o preenchimento da Informação!')

⚠️ INFORMAÇÃO IMPORTANTE SOBRE TRATAMENTO DOS DADOS!!
    Clube: Fluminense   | Data da Partida: 01/11/2024
    Clube: Grêmio       | Data da Partida: 01/11/2024
    Clube: Atlético-MG  | Data da Partida: 07/12/2024
    Clube: Athletico-PR | Data da Partida: 07/12/2024
    Motivo: Partidas não computadas na base original. Confirmado em fonte externa oficial.
    Os valores preenchidos estão na Coluna [Origem_Dado] com o valor: "Input Manual" 
    FONTE: https://fbref.com/pt/partidas/bb1a4828/Fluminense-Gremio-2024Novembro1-Serie-A
    FONTE: https://fbref.com/pt/partidas/fd4de3a4/Atletico-Mineiro-Athletico-Paranaense-2024Dezembro8-Serie-A
✅ Feita o preenchimento da Informação!


#### 🔹Ajuste nos Tipos de Dados

In [0]:
from pyspark.sql.functions import col

df_input_manual = df_input_manual.withColumn("Minutos_em_Campo", col("Minutos_em_Campo").cast("double")) \
    .withColumn("Toques_na_Bola", col("Toques_na_Bola").cast("double")) \
    .withColumn("Desarmes", col("Desarmes").cast("double")) \
    .withColumn("Bloqueios", col("Bloqueios").cast("double")) \
    .withColumn("Gols_Esperados", col("Gols_Esperados").cast("double")) \
    .withColumn("Assistencias_Esperadas", col("Assistencias_Esperadas").cast("double")) \
    .withColumn("Acoes_de_Criacao", col("Acoes_de_Criacao").cast("integer")) \
    .withColumn("Acoes_de_Gol", col("Acoes_de_Gol").cast("double")) \
    .withColumn("Passes_Completos", col("Passes_Completos").cast("double")) \
    .withColumn("Passes_Tentados", col("Passes_Tentados").cast("double")) \
    .withColumn("Prct_Passes_Certos", col("Prct_Passes_Certos").cast("string")) \
    .withColumn("Passes_Progressivos", col("Passes_Progressivos").cast("double")) \
    .withColumn("Conducoes_de_Bola", col("Conducoes_de_Bola").cast("double")) \
    .withColumn("Conducoes_Progressivas", col("Conducoes_Progressivas").cast("double")) \
    .withColumn("Dribles_Tentados", col("Dribles_Tentados").cast("double")) \
    .withColumn("Dribles_Completos", col("Dribles_Completos").cast("double"))



#### 🔹União da base original com os dados preenchidos manualmente

In [0]:
# Contar registros antes da união
registros_antes = df_bronze.count()
print(f"📌 Registros no df_bronze antes da união: {registros_antes}")

# Unir os DataFrames
df_bronze = df_bronze.unionByName(df_input_manual)

# Contar registros depois da união
registros_depois = df_bronze.count()
print(f"📌 Registros no df_bronze depois da união: {registros_depois}")

# Verificar se houve adição de registros
if registros_depois > registros_antes:
    print("✅ Registros adicionados com sucesso!")
else:
    print("⚠️ Nenhum registro foi adicionado. Precisamos investigar.")

print(f'✅ Dados manuais adicionados com sucesso!')

📌 Registros no df_bronze antes da união: 11682
📌 Registros no df_bronze depois da união: 11746
✅ Registros adicionados com sucesso!
✅ Dados manuais adicionados com sucesso!


## 🔶 Analise Geral do arquivo Tratado (Checagem) 

#### 🔹contagem de valores vazios por coluna

In [0]:
from pyspark.sql.functions import col, lit, concat_ws, monotonically_increasing_id, quarter, when, to_date, sum
import pandas as pd

pd.set_option('display.max_columns', None)  # Exibe todas as colunas
pd.set_option('display.width', 1000)        # Ajusta a largura para evitar cortes

# 1️⃣. Criar um DataFrame com a contagem de valores vazios por coluna
empty_counts = df_bronze.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_bronze.columns])

print("📌 Contagem de valores vazios por coluna:")
empty_counts.toPandas()

📌 Contagem de valores vazios por coluna:


,Jogador,Clube,Numero_Camisa,Nacionalidade,Posicao_em_Campo,idade,Minutos_em_Campo,Gols,Assitencias,Penaltis_Batidos,Penaltis_Tentados,Total_Chutes,Chutes_a_Gol,Cartao_Amarelo,Cartao_Vermelho,Toques_na_Bola,Desarmes,Interceptacoes,Bloqueios,Gols_Esperados,Assistencias_Esperadas,Acoes_de_Criacao,Acoes_de_Gol,Passes_Completos,Passes_Tentados,Prct_Passes_Certos,Passes_Progressivos,Conducoes_de_Bola,Conducoes_Progressivas,Dribles_Tentados,Dribles_Completos,Data,Origem_Dado
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,11,0,0,0,0,2,0,0,0,0,0,0,0


#### 🔹resumo estatístico

In [0]:
# 2️⃣. Exibir resumo estatístico das colunas + contagem de valores vazios
print("📌 Estatísticas do DataFrame:")
# Importando biblioteca do Spark
import pyspark.sql.functions as F
import pandas as pd

# Convertendo para Pandas e exibindo todas as colunas
pd.set_option('display.max_columns', None)
df_bronze.summary("count", "min", "max").toPandas()


📌 Estatísticas do DataFrame:


,summary,Jogador,Clube,Numero_Camisa,Nacionalidade,Posicao_em_Campo,idade,Minutos_em_Campo,Gols,Assitencias,Penaltis_Batidos,Penaltis_Tentados,Total_Chutes,Chutes_a_Gol,Cartao_Amarelo,Cartao_Vermelho,Toques_na_Bola,Desarmes,Interceptacoes,Bloqueios,Gols_Esperados,Assistencias_Esperadas,Acoes_de_Criacao,Acoes_de_Gol,Passes_Completos,Passes_Tentados,Prct_Passes_Certos,Passes_Progressivos,Conducoes_de_Bola,Conducoes_Progressivas,Dribles_Tentados,Dribles_Completos,Origem_Dado
0,count,11746,11746,11746,11746,11746,11746,11746,11746,11746,11746,11746,11746,11746,11746,11746,11746,11746,11746,11746,11731,11735,11746,11746,11746,11746,11744,11746,11746,11746,11746,11746,11746
1,min,Abner,Athletico-PR,1,ANG,AM,16,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Input Manual
2,max,Óscar Romero,Vasco da Gama,200,VEN,"WB,RW",44-032,90.0,3,2,2,2,9,6,2,1,143.0,12.0,8,7.0,1.7,1.3,14.0,4.0,122.0,130.0,991.0,27.0,106.0,17.0,16.0,10.0,Original


#### 🔹Ajuste nos Tipos de Dados

In [0]:
from pyspark.sql.functions import col

df_bronze = df_bronze.withColumn("Prct_Passes_Certos", col("Prct_Passes_Certos").cast("double"))


## 💾 Salvar os dados na camada Silver (Parquet no S3)

#### 🔹 Salvando o arquivo no S3 camada Silver

In [0]:
# Caminho correto no S3 para armazenar os arquivos
silver_parquet_path = f"s3a://{aws_bucket_name}/silver/parquet/brasil_seriea_2024_estatistica_jogador"
silver_delta_path = f"s3a://{aws_bucket_name}/silver/delta/brasil_seriea_2024_estatistica_jogador"

# Remover o Delta anterior (use com cautela)
dbutils.fs.rm(silver_delta_path, recurse=True)

# Salvar em Parquet (para acessibilidade geral)
df_bronze.write.mode("overwrite").parquet(silver_parquet_path)

# Salvar em Delta (para controle e governança)
df_bronze.write.format("delta").mode("overwrite").save(silver_delta_path)

print("✅ Dados salvos corretamente em Parquet e Delta, em pastas separadas!")

✅ Dados salvos corretamente em Parquet e Delta, em pastas separadas!


#### 🔹 Salvando o arquivo no DBFS

In [0]:
# Caminho no DBFS para armazenar os dados Silver temporariamente
dbfs_path_silver_estatisticas = "dbfs:/mnt/silver_temp/estatisticas"

# Salvar no DBFS em formato Parquet
df_bronze.write.mode("overwrite").parquet(dbfs_path_silver_estatisticas)

print("✅ Dados transformados e salvos na camada Silver no DBFS com sucesso!")

✅ Dados transformados e salvos na camada Silver no DBFS com sucesso!
